# IBM Data Science Professional Certificate | Applied Data Science Capstone

## Segmenting and Clustering Neighborhoods in Toronto

### Part 3 - Explore and Cluster the Neighborhoods in Toronto

In Part 1 of this Project, we had successfully scraped the Wikipedia page, wrangled the data, cleaned it, and then read it into a pandas dataframe so that it is in a structured format.

In Part 2 of this Project, we had successfully fetched the latitude and the longitude coordinates of each neighborhood using the geospatial data.

Now we try to Explore and cluster the neighborhoods in Toronto. We work with only Boroughs that contain the word Toronto and then replicate the same analysis to all other neighborhoods.Finally clustering is carried out by using K MeansAlgorithm and the clusters are plotted using the Folium Library on a map.

##### Import necessary Libraries

In [115]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

! pip install folium==0.5.0
import folium # plotting library

print('Libraries Imported.')

Libraries Imported.


##### Importing the csv file conatining the Postal Code, Borough, Neighborhood, Latitude and Longitude created in Part-2

In [116]:
# Importing Neighborhood Data from Part-2
df=pd.read_csv('Segmenting and Clustering Neighborhoods in Toronto_Part2.csv')
df.rename(columns={'Neighbourhood':'Neighborhood'},inplace=True)
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


##### Getting all the rows from the DataFrame with Boroughs that contain the word Toronto 

In [117]:
toronto_df = df[df['Borough'].str.contains('Toronto',regex=False)]
toronto_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


#### Fetching the Geographical Coordinates of Toronto for Visualization using Folium

In [118]:
# Defining an instance of geocoder using an user_agent as'toronto_explorer'
address = 'Toronto, Ontario, Canada'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'The geograpical coordinates of Toronto are {latitude}, {longitude}.')

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


##### Visualizing the Neighbourhoods of Toronto using Folium

In [119]:
# creating a map of toronto using latitude and longitude values
toronto_map = folium.Map(location=[latitude, longitude],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(toronto_df['Latitude'],toronto_df['Longitude'],toronto_df['Borough'],toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='red',
    fill=True,
    fill_color='#ff6161',
    fill_opacity=0.7,
    parse_html=False).add_to(toronto_map)
toronto_map

##### Exploring the Neighborhoods of Toronto using the Foursquare API

In [120]:
# Defining Foursquare Credentials and Version
CLIENT_ID = 'ELTGDOB5SSK5OABTGUOV3QAQQRE3OUUZBNLXKOVLXQ4TS4DX' # your Foursquare ID
CLIENT_SECRET = 'GVVUTABGZZUQ05R0BK3GKWL01BPGUHLRERT5ZEFNQOBG30T0' # your Foursquare Secret
VERSION = '20200605' # Foursquare API version
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: ELTGDOB5SSK5OABTGUOV3QAQQRE3OUUZBNLXKOVLXQ4TS4DX


In [144]:
# Creating a function to extract the category of the Neighborhoods in Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=5000, LIMIT= 500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [145]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )
toronto_venues.head()

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site
3,"Regent Park, Harbourfront",43.65426,-79.360636,Distillery Sunday Market,43.650075,-79.361832,Farmers Market
4,"Regent Park, Harbourfront",43.65426,-79.360636,Arvo,43.649963,-79.361442,Coffee Shop


In [146]:
# Checking the shape of the Dataframe
toronto_venues.shape

(3900, 7)

In [147]:
# Checking venues returned for each neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,100,100,100,100,100,100
"Brockton, Parkdale Village, Exhibition Place",100,100,100,100,100,100
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",100,100,100,100,100,100
Central Bay Street,100,100,100,100,100,100
Christie,100,100,100,100,100,100
Church and Wellesley,100,100,100,100,100,100
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,100,100,100,100,100,100


In [149]:
# Checking unique categories from all the returned venues
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 157 uniques categories.


##### Analyzing each Neighbourhood of Toronto

In [150]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop(['Neighborhood'],axis=1,inplace=True) 
toronto_onehot.insert(loc=0, column='Neighborhood', value=toronto_venues['Neighborhood'] )
toronto_onehot.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Amphitheater,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,...,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Yoga Studio,Zoo
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [151]:
# Checking the shape of the Dataframe
toronto_onehot.shape

(3900, 157)

In [152]:
# Grouping rows by neighborhood and taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Amphitheater,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,...,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Yoga Studio,Zoo
0,Berczy Park,0.00,0.01,0.00,0.01,0.00,0.01,0.00,0.00,0.03,...,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.01,0.00
1,"Brockton, Parkdale Village, Exhibition Place",0.00,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.02,...,0.00,0.01,0.01,0.00,0.00,0.01,0.00,0.00,0.00,0.00
2,"Business reply mail Processing Centre, South C...",0.00,0.01,0.00,0.00,0.02,0.01,0.02,0.00,0.03,...,0.00,0.00,0.00,0.01,0.01,0.01,0.00,0.00,0.00,0.00
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.00,0.01,0.00,0.01,0.00,0.01,0.00,0.00,0.03,...,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,Central Bay Street,0.00,0.01,0.00,0.01,0.00,0.00,0.01,0.00,0.04,...,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,Christie,0.00,0.00,0.00,0.01,0.00,0.01,0.02,0.02,0.02,...,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.01,0.00,0.00
6,Church and Wellesley,0.00,0.01,0.00,0.01,0.00,0.00,0.01,0.00,0.02,...,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,"Commerce Court, Victoria Hotel",0.00,0.01,0.00,0.01,0.00,0.01,0.00,0.00,0.04,...,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.01,0.00
8,Davisville,0.01,0.00,0.00,0.00,0.00,0.01,0.04,0.02,0.06,...,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.01,0.00
9,Davisville North,0.01,0.00,0.00,0.00,0.00,0.00,0.03,0.02,0.04,...,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00


In [153]:
# Checking the shape of the Dataframe
toronto_grouped.shape

(39, 157)

In [154]:
# Defining a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [155]:
# Creating a new dataframe and display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Park,Café,Sandwich Place,Plaza,Farmers Market,Dessert Shop,Bakery,Hotel,Historic Site
1,"Brockton, Parkdale Village, Exhibition Place",Park,Café,Coffee Shop,Sandwich Place,Pizza Place,Brewery,Dessert Shop,Scenic Lookout,Seafood Restaurant,Farmers Market
2,"Business reply mail Processing Centre, South C...",Park,Coffee Shop,Café,Beach,Brewery,Pizza Place,Bakery,Gastropub,Asian Restaurant,Indian Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Park,Sandwich Place,Farmers Market,Café,Dessert Shop,Bakery,Liquor Store,Seafood Restaurant,Plaza
4,Central Bay Street,Coffee Shop,Park,Farmers Market,Sandwich Place,Bakery,Café,Plaza,Hotel,Theater,Spa


##### Clusting Neighbourhoods

In [156]:
# Run k-means to cluster the neighborhood into 5 clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 3, 4, 0, 0, 3, 0, 0, 1, 1])

In [157]:
# Creating a new dataframe that includes the cluster as well as the top 10 venues 
neighborhoods_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)

toronto_merged = toronto_df

# merge toronto_grouped with toronto_df to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Café,French Restaurant,Farmers Market,Hotel,Plaza,Dessert Shop,Diner,Pizza Place
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Park,Café,Sandwich Place,Farmers Market,Bakery,Plaza,Spa,Theater,Pizza Place
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Park,Farmers Market,Café,Restaurant,Sandwich Place,Plaza,Pizza Place,Gastropub,Historic Site
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Park,Farmers Market,Bakery,Café,Plaza,Hotel,Dessert Shop,Sandwich Place,Diner
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Park,Coffee Shop,Beach,Café,Bakery,Brewery,Gastropub,Breakfast Spot,Ice Cream Shop,Pizza Place


##### Visualizing the resulting Clusters

In [160]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.gist_rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

##### Examining the Clusters

In [161]:
# Cluster 1
toronto_merged.loc[toronto_merged['Cluster Label'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0,Coffee Shop,Park,Café,French Restaurant,Farmers Market,Hotel,Plaza,Dessert Shop,Diner,Pizza Place
4,Downtown Toronto,0,Coffee Shop,Park,Café,Sandwich Place,Farmers Market,Bakery,Plaza,Spa,Theater,Pizza Place
9,Downtown Toronto,0,Coffee Shop,Park,Farmers Market,Café,Restaurant,Sandwich Place,Plaza,Pizza Place,Gastropub,Historic Site
15,Downtown Toronto,0,Coffee Shop,Park,Farmers Market,Bakery,Café,Plaza,Hotel,Dessert Shop,Sandwich Place,Diner
20,Downtown Toronto,0,Coffee Shop,Park,Café,Sandwich Place,Plaza,Farmers Market,Dessert Shop,Bakery,Hotel,Historic Site
24,Downtown Toronto,0,Coffee Shop,Park,Farmers Market,Sandwich Place,Bakery,Café,Plaza,Hotel,Theater,Spa
30,Downtown Toronto,0,Coffee Shop,Park,Bakery,Café,Farmers Market,Dessert Shop,Plaza,Sandwich Place,Hotel,Pizza Place
36,Downtown Toronto,0,Coffee Shop,Park,Café,Farmers Market,Bakery,Plaza,Dessert Shop,Sandwich Place,Caribbean Restaurant,Historic Site
42,Downtown Toronto,0,Coffee Shop,Park,Café,Farmers Market,Bakery,Dessert Shop,Sandwich Place,Plaza,Pizza Place,Gastropub
48,Downtown Toronto,0,Coffee Shop,Park,Farmers Market,Bakery,Dessert Shop,Sandwich Place,Plaza,Café,Hotel,Mediterranean Restaurant


In [162]:
# Cluster 2
toronto_merged.loc[toronto_merged['Cluster Label'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Central Toronto,1,Coffee Shop,Café,Italian Restaurant,Park,Bakery,Sushi Restaurant,Dog Run,Burger Joint,Pub,Grocery Store
62,Central Toronto,1,Café,Park,Italian Restaurant,Coffee Shop,Grocery Store,Bakery,Indian Restaurant,Brewery,Liquor Store,BBQ Joint
67,Central Toronto,1,Park,Coffee Shop,Café,Grocery Store,Italian Restaurant,Bakery,Supermarket,BBQ Joint,Shopping Mall,Indian Restaurant
68,Central Toronto,1,Park,Café,Coffee Shop,Grocery Store,Italian Restaurant,Bakery,Ice Cream Shop,Sandwich Place,Mexican Restaurant,South American Restaurant
73,Central Toronto,1,Park,Coffee Shop,Café,Italian Restaurant,Bakery,Grocery Store,Liquor Store,Indian Restaurant,BBQ Joint,French Restaurant
79,Central Toronto,1,Park,Italian Restaurant,Café,Bakery,Coffee Shop,BBQ Joint,Bookstore,Indian Restaurant,Grocery Store,Supermarket
83,Central Toronto,1,Park,Café,Coffee Shop,Italian Restaurant,Spa,Indian Restaurant,Greek Restaurant,Grocery Store,BBQ Joint,Ice Cream Shop
86,Central Toronto,1,Park,Café,Coffee Shop,Italian Restaurant,Grocery Store,Spa,Gastropub,Sandwich Place,Farmers Market,Bagel Shop


In [163]:
# Cluster 3
toronto_merged.loc[toronto_merged['Cluster Label'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,West Toronto,2,Café,Coffee Shop,Park,Bar,Bakery,Brewery,Restaurant,Italian Restaurant,Japanese Restaurant,Cocktail Bar
69,West Toronto,2,Café,Park,Coffee Shop,Bakery,Italian Restaurant,Brewery,Bar,Pizza Place,Ice Cream Shop,Indian Restaurant
75,West Toronto,2,Café,Park,Coffee Shop,Bakery,Brewery,Italian Restaurant,Pizza Place,Ice Cream Shop,Sandwich Place,Indian Restaurant
81,West Toronto,2,Café,Coffee Shop,Park,Bakery,Brewery,Italian Restaurant,Ice Cream Shop,Bar,Scenic Lookout,Restaurant


In [164]:
# Cluster 4
toronto_merged.loc[toronto_merged['Cluster Label'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Downtown Toronto,3,Café,Park,Coffee Shop,Sandwich Place,Gastropub,Pizza Place,Italian Restaurant,Grocery Store,Spa,Brewery
37,West Toronto,3,Park,Café,Coffee Shop,Sandwich Place,Pizza Place,Brewery,Farmers Market,Dessert Shop,Bakery,BBQ Joint
43,West Toronto,3,Park,Café,Coffee Shop,Sandwich Place,Pizza Place,Brewery,Dessert Shop,Scenic Lookout,Seafood Restaurant,Farmers Market
74,Central Toronto,3,Coffee Shop,Café,Park,Sandwich Place,Pizza Place,Grocery Store,Spa,Bakery,Theater,South American Restaurant
80,Downtown Toronto,3,Coffee Shop,Café,Park,Sandwich Place,Pizza Place,Spa,Bakery,Farmers Market,Brewery,Grocery Store
84,Downtown Toronto,3,Coffee Shop,Park,Café,Sandwich Place,Farmers Market,Bakery,Liquor Store,Grocery Store,Dessert Shop,Gastropub
91,Downtown Toronto,3,Coffee Shop,Park,Café,Italian Restaurant,Farmers Market,Historic Site,Bakery,Spa,French Restaurant,Restaurant


In [166]:
# Cluster 5
toronto_merged.loc[toronto_merged['Cluster Label'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,East Toronto,4,Park,Coffee Shop,Beach,Café,Bakery,Brewery,Gastropub,Breakfast Spot,Ice Cream Shop,Pizza Place
41,East Toronto,4,Park,Coffee Shop,Café,Greek Restaurant,Brewery,Pizza Place,Historic Site,Farmers Market,Gastropub,Bakery
47,East Toronto,4,Park,Coffee Shop,Café,Beach,Brewery,Gastropub,Bakery,Pizza Place,BBQ Joint,Sushi Restaurant
54,East Toronto,4,Coffee Shop,Park,Café,Brewery,Greek Restaurant,Farmers Market,Beach,Italian Restaurant,Bakery,Gastropub
100,East Toronto,4,Park,Coffee Shop,Café,Beach,Brewery,Pizza Place,Bakery,Gastropub,Asian Restaurant,Indian Restaurant
